In [16]:
import pandas as pd
import numpy as np

POLICY_CSV = "../data/raw/freMTPL2freq.csv"  # IDpol, ClaimNb, Exposure, VehPower, VehAge, DrivAge, BonusMalus, VehBrand, VehGas, Area, Density, Region

CLAIMS_CSV = "../data/raw/freMTPL2sev.csv"   # IDpol, ClaimAmount

def read_any_csv(path):
    
    return pd.read_csv(path, sep=None, engine="python")

pol = read_any_csv(POLICY_CSV)
clm = read_any_csv(CLAIMS_CSV)

print("policy shape:", pol.shape, '\n')
print("claims shape:", clm.shape, '\n')
print("policy cols:", list(pol.columns), '\n')
print("claims cols:", list(clm.columns), '\n')

print(pol.head(3), 
      clm.head(3))



policy shape: (678013, 12) 

claims shape: (26639, 2) 

policy cols: ['IDpol', 'ClaimNb', 'Exposure', 'VehPower', 'VehAge', 'DrivAge', 'BonusMalus', 'VehBrand', 'VehGas', 'Area', 'Density', 'Region'] 

claims cols: ['IDpol', 'ClaimAmount'] 

   IDpol  ClaimNb  Exposure  VehPower  VehAge  DrivAge  BonusMalus VehBrand  \
0    1.0        1      0.10         5       0       55          50      B12   
1    3.0        1      0.77         5       0       55          50      B12   
2    5.0        1      0.75         6       2       52          50      B12   

    VehGas Area  Density       Region  
0  Regular    D     1217  Rhone-Alpes  
1  Regular    D     1217  Rhone-Alpes  
2   Diesel    B       54     Picardie        IDpol  ClaimAmount
0     1552       995.20
1  1010996      1128.12
2  4024277      1851.11


In [14]:
pol.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 678013 entries, 0 to 678012
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   IDpol       678013 non-null  float64
 1   ClaimNb     678013 non-null  int64  
 2   Exposure    678013 non-null  float64
 3   VehPower    678013 non-null  int64  
 4   VehAge      678013 non-null  int64  
 5   DrivAge     678013 non-null  int64  
 6   BonusMalus  678013 non-null  int64  
 7   VehBrand    678013 non-null  object 
 8   VehGas      678013 non-null  object 
 9   Area        678013 non-null  object 
 10  Density     678013 non-null  int64  
 11  Region      678013 non-null  object 
dtypes: float64(2), int64(6), object(4)
memory usage: 62.1+ MB


In [17]:
# 1) Prüfen, ob die float-IDs wirklich "ganzzahlig" 
diff = np.abs(pol["IDpol"] - np.round(pol["IDpol"]))
max_diff = diff.max()
print("größte Abweichung von einer ganzen Zahl:", max_diff)

if max_diff > 0:
    # Zeig ein paar problematische IDs
    offenders = pol.loc[diff > 0, "IDpol"].head(10).tolist()
    raise ValueError(f"IDpol enthält Nicht-Integer-Werte, Beispiele: {offenders}")

# 2) Wenn integer-tauglich: sauber nach 'Int64' (nullable Integer) casten
pol["IDpol"] = np.round(pol["IDpol"]).astype("Int64")
pol["ClaimNb"] = pd.to_numeric(pol["ClaimNb"], errors="raise").astype("Int64")
pol["Exposure"] = pd.to_numeric(pol["Exposure"], errors="raise").astype("float")

print(pol.dtypes)
pol.head(3)

größte Abweichung von einer ganzen Zahl: 0.0
IDpol           Int64
ClaimNb         Int64
Exposure      float64
VehPower        int64
VehAge          int64
DrivAge         int64
BonusMalus      int64
VehBrand       object
VehGas         object
Area           object
Density         int64
Region         object
dtype: object


,IDpol,ClaimNb,Exposure,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Area,Density,Region
0,1,1,0.10,5,0,55,50,B12,Regular,D,1217,Rhone-Alpes
1,3,1,0.77,5,0,55,50,B12,Regular,D,1217,Rhone-Alpes
2,5,1,0.75,6,2,52,50,B12,Diesel,B,54,Picardie


In [18]:
# 1) Objektspalten zu Kategorien (RAM sparen, schnellere .groupby/.merge)
for c in ["VehBrand", "VehGas", "Area", "Region"]:
    if c in pol.columns:
        pol[c] = pol[c].astype("category")

# 2) Weitere numerische Spalten sauber typisieren (lassen wir bewusst als signed ints/floats)
for c in ["VehPower", "VehAge", "DrivAge", "BonusMalus", "Density"]:
    if c in pol.columns:
        pol[c] = pd.to_numeric(pol[c], errors="raise")

# 3) Harte Plausibilitäten (nur zählen, nicht crashen)
issues = {
    "Exposure<0": int((pol["Exposure"] < 0).sum()),
    "ClaimNb<0": int((pol["ClaimNb"] < 0).sum()),
    "ClaimNb_not_integer": int(((pol["ClaimNb"].astype("float") % 1) != 0).sum()),
}
issues


{'Exposure<0': 0, 'ClaimNb<0': 0, 'ClaimNb_not_integer': 0}

In [19]:
pol.info(memory_usage="deep")

import os
os.makedirs("data/work", exist_ok=True)
pol.to_parquet("data/work/freMTPL2freq_clean.parquet", index=False)
print("gespeichert: data/work/freMTPL2freq_clean.parquet")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 678013 entries, 0 to 678012
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   IDpol       678013 non-null  Int64   
 1   ClaimNb     678013 non-null  Int64   
 2   Exposure    678013 non-null  float64 
 3   VehPower    678013 non-null  int64   
 4   VehAge      678013 non-null  int64   
 5   DrivAge     678013 non-null  int64   
 6   BonusMalus  678013 non-null  int64   
 7   VehBrand    678013 non-null  category
 8   VehGas      678013 non-null  category
 9   Area        678013 non-null  category
 10  Density     678013 non-null  int64   
 11  Region      678013 non-null  category
dtypes: Int64(2), category(4), float64(1), int64(5)
memory usage: 45.3 MB
gespeichert: data/work/freMTPL2freq_clean.parquet


In [25]:
# 1) IDpol integer-tauglich?
diff = np.abs(clm["IDpol"] - np.round(clm["IDpol"]))
max_diff = diff.max()
print("clm: größte Abweichung von einer ganzen Zahl:", max_diff)
if max_diff > 0:
    offenders = clm.loc[diff > 0, "IDpol"].head(10).tolist()
    raise ValueError(f"sev.IDpol enthält Nicht-Integer-Werte, Beispiele: {offenders}")

clm["IDpol"] = np.round(clm["IDpol"]).astype("Int64")

# 2) ClaimAmount robust numerisch (NA erlaubt)
clm["ClaimAmount"] = pd.to_numeric(clm["ClaimAmount"], errors="coerce").astype("float")

# 3) Basiskontrollen
issues_clm = {
    "ClaimAmount<0": int((clm["ClaimAmount"] < 0).sum()),
    "IDpol_NA": int(clm["IDpol"].isna().sum()),
    "ClaimAmount_NA": int(clm["ClaimAmount"].isna().sum()),
}
issues_clm


sev: größte Abweichung von einer ganzen Zahl: 0


{'ClaimAmount<0': 0, 'IDpol_NA': 0, 'ClaimAmount_NA': 0}

In [27]:
clm.to_parquet("data/work/freMTPL2sev_clean.parquet", index=False)
print("gespeichert: data/work/freMTPL2sev_clean.parquet")

gespeichert: data/work/freMTPL2sev_clean.parquet


In [37]:
FREQ_PARQ = "../data/work/freMTPL2freq_clean.parquet"
CLAIM_PARQ = "../data/work/freMTPL2sev_clean.parquet"

freq = pd.read_parquet(FREQ_PARQ)
clm = pd.read_parquet(CLAIM_PARQ)

freq.shape, clm.shape, list(freq.columns), list(clm.columns)

((678013, 12),
 (26639, 2),
 ['IDpol',
  'ClaimNb',
  'Exposure',
  'VehPower',
  'VehAge',
  'DrivAge',
  'BonusMalus',
  'VehBrand',
  'VehGas',
  'Area',
  'Density',
  'Region'],
 ['IDpol', 'ClaimAmount'])

In [40]:
dup_clm = clm["IDpol"].duplicated(keep=False).sum()
dup_clm #3068 doppelte IDpol bei claims > aggregieren


np.int64(3068)

In [42]:
clm_agg = clm.groupby("IDpol", as_index=False)["ClaimAmount"].sum()
clm_agg.shape


(24950, 2)

In [51]:
#left join 

df = freq.merge(clm_agg, on="IDpol", how="left", validate="one_to_one")

if "ClaimAmount" in df:
    df["ClaimAmount"] = df["ClaimAmount"].fillna(0.0)


df.shape, df.isna().sum().sort_values(ascending=False)

((678013, 13),
 IDpol          0
 ClaimNb        0
 Exposure       0
 VehPower       0
 VehAge         0
 DrivAge        0
 BonusMalus     0
 VehBrand       0
 VehGas         0
 Area           0
 Density        0
 Region         0
 ClaimAmount    0
 dtype: int64)

In [52]:
N = len(df)

tail = {
    "N": N,
    "ClaimNb>4_count": int((df["ClaimNb"] > 4).sum()),
    "ClaimNb>4_pct": float((df["ClaimNb"] > 4).mean()),
    "Exposure>1_count": int((df["Exposure"] > 1).sum()),
    "Exposure>1_pct": float((df["Exposure"] > 1).mean()),
    "DrivAge>90_count": int((df["DrivAge"] > 90).sum()),
    "DrivAge>90_pct": float((df["DrivAge"] > 90).mean()),
    "VehAge>40_count": int((df["VehAge"] > 40).sum()),
    "VehAge>40_pct": float((df["VehAge"] > 40).mean()),
}
tail

{'N': 678013,
 'ClaimNb>4_count': 9,
 'ClaimNb>4_pct': 1.3274081765393879e-05,
 'Exposure>1_count': 1224,
 'Exposure>1_pct': 0.0018052751200935675,
 'DrivAge>90_count': 401,
 'DrivAge>90_pct': 0.0005914340875469939,
 'VehAge>40_count': 230,
 'VehAge>40_pct': 0.00033922653400451024}

In [53]:
# ClaimNb: oberes Ende der Skala
claimnb_top = df["ClaimNb"].value_counts().sort_index(ascending=False).head(10)
print("ClaimNb (Top Werte):\n", claimnb_top, "\n")

# Exposure: Quantile + Max
exp_stats = df["Exposure"].describe(percentiles=[0.5, 0.75, 0.9, 0.95, 0.99])
print("Exposure-Stats:\n", exp_stats, "\n")

# DrivAge/VehAge: Quantile + Max
driv_stats = df["DrivAge"].describe(percentiles=[0.5, 0.75, 0.9, 0.95, 0.99])
veh_stats  = df["VehAge"].describe(percentiles=[0.5, 0.75, 0.9, 0.95, 0.99])
print("DrivAge-Stats:\n", driv_stats, "\n")
print("VehAge-Stats:\n",  veh_stats,  "\n")


ClaimNb (Top Werte):
 ClaimNb
16        1
11        3
9         1
8         1
6         1
5         2
4         7
3        82
2      1784
1     32178
Name: count, dtype: Int64 

Exposure-Stats:
 count    678013.000000
mean          0.528750
std           0.364442
min           0.002732
50%           0.490000
75%           0.990000
90%           1.000000
95%           1.000000
99%           1.000000
max           2.010000
Name: Exposure, dtype: float64 

DrivAge-Stats:
 count    678013.000000
mean         45.499122
std          14.137444
min          18.000000
50%          44.000000
75%          55.000000
90%          65.000000
95%          72.000000
99%          80.000000
max         100.000000
Name: DrivAge, dtype: float64 

VehAge-Stats:
 count    678013.000000
mean          7.044265
std           5.666232
min           0.000000
50%           6.000000
75%          11.000000
90%          15.000000
95%          17.000000
99%          21.000000
max         100.000000
Name: VehAge, dtype

In [54]:
if (df["Exposure"] > 1).any():
    over1 = df.loc[df["Exposure"] > 1, "Exposure"].describe(percentiles=[0.95, 0.99])
    print("Exposure > 1 – Details:\n", over1)
else:
    print("Keine Exposure-Werte > 1 gefunden.")


Exposure > 1 – Details:
 count    1224.000000
mean        1.113840
std         0.159296
min         1.010000
50%         1.040000
95%         1.460000
99%         1.803900
max         2.010000
Name: Exposure, dtype: float64


In [64]:
#kappung wie in Schelldorfer & Wüthrich (2019)


df_cap = df.copy()

# ClaimNb <= 4
if "ClaimNb" in df_cap:
    df_cap["ClaimNb"] = np.minimum(df_cap["ClaimNb"], 4)

# Exposure <= 1
if "Exposure" in df_cap:
    df_cap["Exposure"] = np.minimum(df_cap["Exposure"], 1.0)

# DrivAge <= 90, VehAge <= 40
if "DrivAge" in df_cap:
    df_cap["DrivAge"] = np.minimum(df_cap["DrivAge"], 90)
if "VehAge" in df_cap:
    df_cap["VehAge"] = np.minimum(df_cap["VehAge"], 40)


In [57]:
#sanity check

changed_counts = {}

#  vergleichen gegen das ursprüngliche df (
for col, cap in [("ClaimNb", 4), ("Exposure", 1.0), ("DrivAge", 90), ("VehAge", 40)]:
    if col in df_cap and col in df:
        changed_counts[f"{col}_changed"] = int((df_cap[col] != df[col]).sum())
        changed_counts[f"{col}_max_after"] = float(df_cap[col].max())
        changed_counts[f"{col}_max_before"] = float(df[col].max())
changed_counts


{'ClaimNb_changed': 9,
 'ClaimNb_max_after': 4.0,
 'ClaimNb_max_before': 16.0,
 'Exposure_changed': 1224,
 'Exposure_max_after': 1.0,
 'Exposure_max_before': 2.01,
 'DrivAge_changed': 401,
 'DrivAge_max_after': 90.0,
 'DrivAge_max_before': 100.0,
 'VehAge_changed': 230,
 'VehAge_max_after': 40.0,
 'VehAge_max_before': 100.0}

In [58]:
df_cap.to_parquet("../data/work/freMTPL2_joined_capped.parquet", index=False)
"gespeichert: data/work/freMTPL2_joined_capped.parquet"


'gespeichert: data/work/freMTPL2_joined_capped.parquet'